In [1]:
# Statistical testing of differences between max / min differences for each combination of scaling transformation and dataset
import pandas as pd
from scipy.stats import wilcoxon, normaltest
from pathlib import Path
import re
datasets = ["svd", "voiced"]

filename_list = list([file.name for file in Path("svd_results").glob("*.json")])

results_table = pd.DataFrame(data=[], columns=["dataset", "transformer", "model", "p_value"])
for dataset in datasets:
    for name in filename_list:
        model = re.split(r"_[0-9 ]+_", name.split(".")[0])[1]
        transformer = name.split("_")[1]
        path_random = Path(f"{dataset}_results", name)
        path_stratified = Path(f"{dataset}_results_stratified", name)
        data_random = pd.read_json(path_random).transpose()
        data_stratified = pd.read_json(path_stratified).transpose()
        data_random = pd.json_normalize(data_random["bcc"])
        data_stratified = pd.json_normalize(data_stratified["bcc"])
        data_random["diff"] = data_random["leakage"] - data_random["correct"]
        data_stratified["diff"] = data_stratified["leakage"] - data_stratified["correct"]

        for split, data in zip(["random", "stratified"], [data_random, data_stratified]):
            # Test of normality - are the differences drawn from a normal distribution?
            _, p_value = normaltest(data["diff"])
            print(f"{dataset:<6} {split:<10} {transformer:<20} {model:<20} {p_value:.3f}")
        # Test of similarity, do the pairs of differences for random and stratified split have the same mean value?
        # _, p_value = wilcoxon(data_random["diff"], data_stratified["diff"])
        # results_table.loc[results_table.shape[0], :] = [dataset, transformer, model, p_value]

# No test results are consistent so we cannot decide on the normality of data or the differences in results between the random split and stratified split

C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff

svd    random     MaxAbsScaler         adaboost             nan
svd    stratified MaxAbsScaler         adaboost             nan
svd    random     MaxAbsScaler         dt                   nan
svd    stratified MaxAbsScaler         dt                   nan
svd    random     MaxAbsScaler         gaussianNB           nan
svd    stratified MaxAbsScaler         gaussianNB           nan
svd    random     MaxAbsScaler         gaussian_process     nan
svd    stratified MaxAbsScaler         gaussian_process     nan
svd    random     MaxAbsScaler         knn                  nan
svd    stratified MaxAbsScaler         knn                  nan
svd    random     MaxAbsScaler         lda                  nan
svd    stratified MaxAbsScaler         lda                  nan
svd    random     MaxAbsScaler         mlp                  nan
svd    stratified MaxAbsScaler         mlp                  nan
svd    random     MaxAbsScaler         qda                  nan
svd    stratified MaxAbsScaler         q

C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff

svd    random     RobustScaler         mlp                  nan
svd    stratified RobustScaler         mlp                  nan
svd    random     RobustScaler         qda                  nan
svd    stratified RobustScaler         qda                  nan
svd    random     RobustScaler         rf                   nan
svd    stratified RobustScaler         rf                   nan
svd    random     RobustScaler         svm                  nan
svd    stratified RobustScaler         svm                  nan
svd    random     StandardScaler       adaboost             nan
svd    stratified StandardScaler       adaboost             nan
svd    random     StandardScaler       dt                   nan
svd    stratified StandardScaler       dt                   nan
svd    random     StandardScaler       gaussianNB           nan
svd    stratified StandardScaler       gaussianNB           nan
svd    random     StandardScaler       gaussian_process     nan
svd    stratified StandardScaler       g

C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff"])
C:\Users\stein\AppData\Local\Temp\ipykernel_9076\3717769343.py:26: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_value = normaltest(data["diff

In [2]:
# Testing the significancy of mean difference. We assume the results do not follow normal distribution.
import pandas as pd
import numpy as np
from scipy.stats import permutation_test
from pathlib import Path

# Metric used for determining the difference under null hypothesis
# (in this case, is the null hypothesis states the mean value of the samples is same)
stat_func = lambda a, b: np.mean(a - b)
# list of dataset names
datasets = ["svd", "voiced"]
# list of files - it contains the combination of scaling transformation and ML model
filename_list = list([file.name for file in Path("svd_results").glob("*.json")])

# results table to save the results
results_table = pd.DataFrame(data=[], columns=["dataset", "transformer", "model", "split", "p_value"])
# Iterating through SVD and VOICED
for dataset in datasets:
    # Iterating through the file names to get all transformation - model combinations
    for name in filename_list:
        model = re.split(r"_[0-9 ]+_", name.split(".")[0])[1] # ML model name from the file name
        transformer = name.split("_")[1] # Scaling transformation from the file name
        # Setting paths for the results from random splits and stratified splits
        path_random = Path(f"{dataset}_results", name)
        path_stratified = Path(f"{dataset}_results_stratified", name)
        # Loading json files related to the specific model and transformation
        data_random = pd.read_json(path_random).transpose()
        data_stratified = pd.read_json(path_stratified).transpose()
        # Extracting the UAR of the leakage and correct results
        data_random = pd.json_normalize(data_random["bcc"])
        data_stratified = pd.json_normalize(data_stratified["bcc"])
        # data_random["diff"] = data_random["leakage"] - data_random["correct"]

        # The test is done separately for the stratified and random splits
        for split, data in zip(["random", "stratified"], [data_random, data_stratified]):
            # Permutation test with the null hypothesis "leakage model has the same mean performance as the correct model"
            # The setting of the test is so that the performance of the leakage and correct models are taken as pair samples
            # since they are not etirely independent (influence of the model settings is to be expected)
            result = permutation_test((data["leakage"], data["correct"]), statistic=stat_func,
                       permutation_type='samples', alternative='two-sided',
                       vectorized=False, n_resamples=10000, random_state=0)
            # Saving the dataset - transformer - model - split type - result info
            # It's a standard statistical test so p-value < 0.05 -> null hypothesis is rejected -> the difference in performance
            # is statistically significant
            results_table.loc[results_table.shape[0], :] = [dataset, transformer, model, split, result.pvalue]
            print(f"{dataset:<6} {split:<10} {transformer:<20} {model:<20} {result.pvalue:.3f}")



svd    random     MaxAbsScaler         adaboost             1.000
svd    stratified MaxAbsScaler         adaboost             1.000
svd    random     MaxAbsScaler         dt                   1.000
svd    stratified MaxAbsScaler         dt                   1.000
svd    random     MaxAbsScaler         gaussianNB           1.000
svd    stratified MaxAbsScaler         gaussianNB           1.000
svd    random     MaxAbsScaler         gaussian_process     0.500
svd    stratified MaxAbsScaler         gaussian_process     0.500
svd    random     MaxAbsScaler         knn                  0.500
svd    stratified MaxAbsScaler         knn                  1.000
svd    random     MaxAbsScaler         lda                  1.000
svd    stratified MaxAbsScaler         lda                  1.000
svd    random     MaxAbsScaler         mlp                  1.000
svd    stratified MaxAbsScaler         mlp                  0.500
svd    random     MaxAbsScaler         qda                  1.000
svd    str

In [3]:
# Saving results
results_table.to_csv("permutation_pair_test_mean.csv", index=False)

In [4]:
# Checking which hypotheses can be rejected and saving the results
results_table = pd.read_csv("permutation_pair_test_mean.csv")
results_table[results_table.p_value < 0.05].groupby(["dataset", "transformer"])["p_value"].count().reset_index().pivot(index="transformer", columns=["dataset"], values="p_value").fillna(0).astype(int).to_csv("pivot_permutation_mean.csv")

In [5]:
# Showing the hypotheses that are to be rejected to mark them in the paper
results_table[results_table.p_value < 0.05]

,dataset,transformer,model,split,p_value
